In [9]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [10]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


**Reshape THE IMAGE TO 1-D vector.**

In [11]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


**USE TensorFlow To Train a Logitstic Regression Model**
    
    1.Input data. Load the training, validation and test data into constants, and attach them to the graph.

    2.We are going to train `weights` and `biases`. 
        -Initialize `weights` using Normal distribution. 
        -Initialize `biases` with all zeros.
    
    3.Generate logits $y = w*x + b$ using functions TensorFlow provides with.

    4.Generate loss function `loss` use TensorFlow provides, the loss function roughly is 
$$loss = \frac{1}{2n}\sum(y_i - y'_i)^2$$

    5.Use Gradent Descent to find the mini value of loss function above.
    
    6. To track the accuracy of our model during trainning, we need to print the training info.

In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
    
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Before we runt the training. Firstly, we set up standard for measuring the performance of our model. The Accuracy, in other word.

In [13]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's run this computation and iterate:
    1. We will iterate 801 times.
    2. We need to start the computation in a session. In other words, we launch the graph in a session.

In [14]:
num_steps = 801


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  print()
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%\n' % accuracy(
        valid_prediction.eval(), valid_labels))

# print the accuracy of our model.
  print()
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Loss at step 0: 15.549465
Training accuracy: 9.0%
Validation accuracy: 12.0%

Loss at step 100: 2.407291
Training accuracy: 71.0%
Validation accuracy: 71.5%

Loss at step 200: 1.923929
Training accuracy: 73.9%
Validation accuracy: 73.4%

Loss at step 300: 1.658597
Training accuracy: 75.5%
Validation accuracy: 74.5%

Loss at step 400: 1.482522
Training accuracy: 76.6%
Validation accuracy: 74.5%

Loss at step 500: 1.354068
Training accuracy: 77.3%
Validation accuracy: 74.5%

Loss at step 600: 1.254774
Training accuracy: 78.0%
Validation accuracy: 75.1%

Loss at step 700: 1.174731
Training accuracy: 78.7%
Validation accuracy: 75.1%

Loss at step 800: 1.108167
Training accuracy: 79.0%
Validation accuracy: 75.1%


Test accuracy: 84.1%
